# Cosmicflows Distance-Velocity API

This notebook explains how you can simply interact with Cosmicflows calculators within your *Python* code.

**Note:** This API can be also queried in any programming language of choice.

- Communication method: POST, GET
- Input data: JSON 

**NAM DV-calculator API:** http://edd.ifa.hawaii.edu/NAMcalculator/api.php

**CF3 DV-calculator API:** http://edd.ifa.hawaii.edu/CF3calculator/api.php

### Copy the following function into your code

The following code snippet provides an interface function to query each of the calculators.

Required python packages:

- requests (to handle the online API requests)
- json (to translate between Python dictionaries and json format)

In [1]:
import requests
import json

def DVcalculator(alpha, delta, system='supergalactic', parameter='distance', value=20, calculator='NAM'):
    
    """
    Inputs: 
        alpha: (float) [deg]
            first coordinate parameter  (RA,  Glon, SGL)
        delta: (float) [deg]
            second coordinate parameter (Dec, Glat, SGB)  
        system: (string)
            coordinate system: 
            Options are:
                "equatorial"
                "galactic"
                "supergalactic"
        parameter: (string)
            the quantity whose value is provided
            Options are:
                "distance"
                "velocity"
        value: (float)
            the value of the input quantity
            distance in [Mpc] and velocity in [km/s]
            
        calcualtor: desired Cosmicflows caluclator
            Options are:
                "NAM" to query the calcualtor at http://edd.ifa.hawaii.edu/NAMcalculator
                "CF3" to query the calcualtor at http://edd.ifa.hawaii.edu/CF3calculator
        
    Output:
        A python dictionary which contains the distance and velocity of the 
        given object and the coordinate of the object in different systems

    """
    
    coordinate = [float(alpha), float(delta)]
    query  = {
              'coordinate': coordinate,
              'system': system,
              'parameter': parameter,
              'value': float(value)
             }
    headers = {'Content-type': 'application/json'}
    
    API_url = 'http://edd.ifa.hawaii.edu/'+calculator+'calculator/api.php'
    
    try:
        r = requests.get(API_url, data=json.dumps(query), headers=headers)
        output = json.loads(r.text) # a python dictionary
    except:
        print("Something went wrong!")  
        print("Please check your intput parameters ...")
        output = None

    return output

## Example 1

### Sending a request to the NAM D-V calcualtor (d < 38 Mpc)
http://edd.ifa.hawaii.edu/NAMcalculator

<img src="./nam_demo.png" style="width: 120%">


**Here is how to send the same request in a Python code:**

    SGL = 102 deg
    SGB = -2 deg
    Coordinate system = supergalactic
    input velocity = 1000 km/s

In [2]:
example_1 = DVcalculator(102, -2, system='supergalactic', parameter='velocity', value=1000, calculator='NAM')

example_1

{'message': 'Success',
 'RA': 187.7891703346409,
 'Dec': 13.333860121247609,
 'Glon': 282.9654677357161,
 'Glat': 75.4136002414933,
 'SGL': 102.0,
 'SGB': -2.0,
 'velocity': 1000.0,
 'distance': [8.08088612690689, 18.786290885088945, 22.097850275812398]}

Here, **example_1** is a Python dictionray. The keys of the output dictionary are self explanatory.

There is an additional key, **message** that holds the message of the backend code that generates the output values. In case of an unsuccessful API, *message* holds the cause of error that helps to correct the mistaken outputs.

In the following cell we show how to extract distance from the output of the calucaltor. Please *note* that the generated distance is always provided as a list, becuase multiple distances can be associated to one radial velocity.

In [3]:
distance_1 = example_1["distance"]

distance_1

[8.08088612690689, 18.786290885088945, 22.097850275812398]

## Example 2


**How to extract the radial velocity of an object with a given distance**

    SGL = 102 deg
    SGB = -2 deg
    Coordinate system = supergalactic
    input distance = 30 Mpc
    Calculator: NAM (http://edd.ifa.hawaii.edu/NAMcalculator)

In [4]:
example_2 = DVcalculator(102, -2, system='supergalactic', parameter='distance', value=30, calculator='NAM')

example_2

{'message': 'Success',
 'RA': 187.7891703346409,
 'Dec': 13.333860121247609,
 'Glon': 282.9654677357161,
 'Glat': 75.4136002414933,
 'SGL': 102.0,
 'SGB': -2.0,
 'velocity': 1790.9019256321444,
 'distance': [30.0]}

## Example 3

### Sending a request to the Cosmicflows-3 D-V calcualtor (d < 200 Mpc)
http://edd.ifa.hawaii.edu/CF3calculator

<img src="./cf3_demo.png" style="width: 75%">


**Here is how to send the same request in a Python code:**

    Glon = 283 deg
    Glat = 75 deg
    Coordinate system = galactic
    input velocity = 9000 km/s

In [5]:
example_3 = DVcalculator(283, 75, system='galactic', parameter='velocity', value=9000, calculator='CF3')

example_3

{'message': 'Success',
 'RA': 187.66534187250852,
 'Dec': 12.938131149621988,
 'Glon': 283.0,
 'Glat': 75.0,
 'SGL': 102.34474779489476,
 'SGB': -2.229027957763543,
 'observed': {'velocity': 9000.0, 'distance': [136.90134347390065]},
 'adjusted': {'velocity': 9000.0, 'distance': [134.26214472224996]}}

### Extracting distance given the observed velocity of $V_{ls}=9000$ km/s

The output distance is always presented in a list.

In [6]:
dist_obs = example_3["observed"]["distance"]

dist_obs

[136.90134347390065]

## Example 4


**How to obtain radial velocity for a given distance**

    RA = 187 deg
    Dec = 13 deg
    Coordinate system = equatorial
    input distance = 180 Mpc
    Calculator: Cosmicflows-3 Distance–Velocity (http://edd.ifa.hawaii.edu/CF3calculator)

In [12]:
example_4 = DVcalculator(187, 13, system='equatorial', parameter='distance', value=180, calculator='CF3')

example_4

{'message': 'Success',
 'RA': 186.99999497668534,
 'Dec': 13.000001835269385,
 'Glon': 280.56811463100814,
 'Glat': 74.84475460794316,
 'SGL': 102.09834270874016,
 'SGB': -2.8320379440905414,
 'observed': {'velocity': 12515.699706446017, 'distance': [180.0]},
 'adjusted': {'velocity': 12940.58481990226, 'distance': [180.0]}}

In [8]:
Vls_observed = example_4["observed"]["velocity"]
Vls_adjusted = example_4["adjusted"]["velocity"]

print("V_ls Observed: %d km/s"%Vls_observed)
print("V_ls Adjusted: %d km/s"%Vls_adjusted)

V_ls Observed: 12515 km/s
V_ls Adjusted: 12940 km/s


# Example 5

### Looping over a list of objects

You can find our example input list (**test_api.csv**) [HERE](https://raw.githubusercontent.com/ekourkchi/Cosmicflows_API/main/test_api.csv).

In [9]:
import pandas as pd
import numpy as np

# reading the list into a pandas dataFrame
df = pd.read_csv("test_api.csv")

# modifying the headers
for col in df.columns:
    newcol = col.split("(")[0].strip()
    df.rename(columns={col:newcol}, inplace=True)
    
df

,id,RA,Dec,Vls
0,27,6.1965,-20.7324,16092
1,76,9.8597,6.7340,12139
2,119,14.0672,-1.2561,13460
3,147,17.1579,2.2684,13163
4,151,17.2130,-15.4072,16064
5,168,18.7404,0.4311,13597
6,189,20.8597,1.7049,9720
7,193,21.2814,8.6992,14676
8,194,21.4952,-1.3394,5495
9,195,21.7281,19.2139,12841


For each object in the list, you have to make a single API call. 
Please make sure that your are feeding the correct values in each of the API calls and you are handling the exceptions properly.

In [10]:
N = len(df)
print("Number of objects: ", N)

ra = df.RA.values    # deg
dec = df.Dec.values  # deg
vls = df.Vls.values  # km/s

distance = np.zeros(N)

# loop over the list
for i in  range(N):
    try: 
        output = DVcalculator(ra[i], dec[i], system='equatorial', parameter='velocity', value=vls[i], calculator='CF3')
        distance[i] = output["observed"]["distance"][0]
    except:
        distance[i] = np.nan

# adding a new column that holds distances in Mpc
df['distance'] = distance

df

Number of objects:  10


,id,RA,Dec,Vls,distance
0,27,6.1965,-20.7324,16092,214.131591
1,76,9.8597,6.7340,12139,159.250496
2,119,14.0672,-1.2561,13460,181.125329
3,147,17.1579,2.2684,13163,175.774712
4,151,17.2130,-15.4072,16064,217.766419
5,168,18.7404,0.4311,13597,183.637349
6,189,20.8597,1.7049,9720,128.972149
7,193,21.2814,8.6992,14676,200.567299
8,194,21.4952,-1.3394,5495,68.783092
9,195,21.7281,19.2139,12841,172.303999


In [11]:
# saving the resulting dataFrame that contains a distance column
df.to_csv('output_api.csv')

A copy of the output file is stored here: [output_api.csv](https://raw.githubusercontent.com/ekourkchi/Cosmicflows_API/main/output_api.csv).

### How to acknowledge this work

If you use the results of this work in your research or other applications, please cite [Kourkchi et al. 2020, AJ, 159, 67](https://ui.adsabs.harvard.edu/abs/2020AJ....159...67K/abstract).